In [2]:
!pip install dash
import dash

In [84]:
import joblib

def load_model():
    # Load the linear regression model from the file
    model = joblib.load('Real Estate Price Prediction_model.joblib') 
    return model


In [86]:
# Load the linear regression model
model = load_model()

In [88]:
from dash import html, dcc, Input, Output, State 
import pandas as pd 

In [90]:
#Initialize the Dash app
app = dash.Dash(__name__)

In [92]:
# Import necessary libraries
import dash
from dash import dcc, html

# Define the layout of the app
app.layout = html.Div(
    [
        html.Div(
            [
                html.H1("Real Estate Price Prediction", style={"text-align": "center", "color": "#007BFF"}),
                html.Div(
                    [
                        dcc.Input(
                            id="distance_to_mrt",
                            type="number",
                            placeholder="Distance to MRT Station (meters)",
                            style={"margin": "10px", "padding": "10px", "border-radius": "5px"},
                        ),
                        dcc.Input(
                            id="num_convenience_stores",
                            type="number",
                            placeholder="Number of Convenience Stores",
                            style={"margin": "10px", "padding": "10px", "border-radius": "5px"},
                        ),
                        dcc.Input(
                            id="latitude",
                            type="number",
                            placeholder="Latitude",
                            style={"margin": "10px", "padding": "10px", "border-radius": "5px"},
                        ),
                        dcc.Input(
                            id="longitude",
                            type="number",
                            placeholder="Longitude",
                            style={"margin": "10px", "padding": "10px", "border-radius": "5px"},
                        ),
                        html.Button(
                            "Predict Price",
                            id="predict_button",
                            n_clicks=0,
                            style={
                                "margin": "10px",
                                "padding": "10px",
                                "background-color": "#007BFF",
                                "color": "white",
                                "border": "none",
                                "border-radius": "5px",
                                "cursor": "pointer",
                            },
                        ),
                    ],
                    style={"text-align": "center"},
                ),
                html.Div(
                    id="prediction_output",
                    style={"text-align": "center", "font-size": "20px", "margin-top": "20px", "color": "#007BFF"},
                ),
            ],
            style={
                "width": "50%",
                "margin": "0 auto",
                "border": "2px solid #007BFF",
                "padding": "20px",
                "border-radius": "10px",
                "background-color": "#f8f9fa",
                "box-shadow": "0 4px 8px rgba(0, 0, 0, 0.1)",
            },
        )
    ],
    style={"background-color": "#f0f8ff", "height": "100vh", "display": "flex", "justify-content": "center", "align-items": "center"},
)


In [94]:
# Define callback to update output
@app.callback(
    Output('prediction_output', 'children'),
    [Input('predict_button', 'n_clicks')],
    [State('distance_to_mrt', 'value'), 
     State('num_convenience_stores', 'value'),
     State('latitude', 'value'),
     State('longitude', 'value')]
)
def update_output(n_clicks, distance_to_mrt, num_convenience_stores, latitude, longitude):
    try:
        if n_clicks > 0 and all(v is not None for v in [distance_to_mrt, num_convenience_stores, latitude, longitude]):
            # Prepare the feature vector
            features = pd.DataFrame([[distance_to_mrt, num_convenience_stores, latitude, longitude]], 
                                    columns=['Distance to the nearest MRT station', 
                                             'Number of convenience stores', 
                                             'Latitude', 
                                             'Longitude'])
            
            # Print debug information
            print("Features:")
            print(features)

            # Predict using your machine learning model
            prediction = model.predict(features)[0]

            # Ensure prediction is a scalar value
            prediction_scalar = prediction.item() if hasattr(prediction, 'item') else prediction

            # Format the prediction for display
            formatted_prediction = f'Predicted House Price of Unit Area: {prediction_scalar:.2f}'
            
            return formatted_prediction
        elif n_clicks > 0:
            return 'Please enter all values to get a prediction'
        return ''
    except Exception as e:
        print(f"Error in callback: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return f'Error occurred: {str(e)}'


In [96]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)